In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_FLEET_SIZE,
)
from modules.stochastic_program.factory import StochasticProgramFactory



In [3]:
%load_ext autoreload
%autoreload 2

# Prepare Model Parameters
## Scenario Data
Make scenarios complete cross product

In [4]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [5]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [6]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [7]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [8]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [9]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [10]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Other Model Parameters


In [11]:
distances = pd.read_pickle(PATH_DISTANCES)

In [12]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [13]:
real_fleet_size = pd.read_pickle(PATH_FLEET_SIZE).to_dict()['id']

# Create & Solve Stochastic Program

In [14]:
vehicle_types = ALL_VEHICLE_TYPES
factory = StochasticProgramFactory(scenarios, distances.apply(lambda x: round(x, 2)), probabilities, node_df, vehicle_types)
factory.set_initial_allocation(real_fleet_size)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.00 seconds
_convert_demand finished in 0.02 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.03 seconds
_set_max_demand finished in 0.01 seconds
set_initial_allocation finished in 0.01 seconds


In [15]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [16]:
stochastic_program.relocations_disabled = False

In [26]:
stochastic_program.create_model(beta=0.5, alpha=0.75)

_create_variables finished in 0.22 seconds
_create_objectives finished in 0.23 seconds
_create_demand_constraints finished in 0.30 seconds
_create_relocation_binary_constraints finished in 0.01 seconds
_create_big_u_sum_constraints finished in 0.04 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
_create_no_refused_demand_constraints finished in 0.01 seconds
_create_relocations_constraints finished in 0.10 seconds
_create_vehicle_trips_starting_constraints finished in 0.04 seconds
_create_vehicle_trips_ending_constraints finished in 0.04 seconds
_create_initial_allocation_constraints finished in 0.01 seconds
_create_non_anticipativity_constraints finished in 0.10 seconds
_create_value_at_risk_constraints finished in 0.16 seconds
_create_constraints finished in 0.84 seconds
create_model finished in 1.57 seconds


In [27]:
stochastic_program.solve()

Status: Optimal
Optimal Value of Objective Function:  55499.27874999997
solve finished in 28.31 seconds


In [28]:
scenarios.reset_index().nunique()

scenarios          8
start_hex_ids     19
end_hex_ids       19
time               2
vehicle_types      3
demand           142
dtype: int64

In [29]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 0.10 seconds
get_results_by_region_df finished in 0.01 seconds
get_summary finished in 0.16 seconds


{'status': 'Optimal',
 'objective': 55499.27874999997,
 'expected_profit': 55794.337499999936,
 'eta': 55204.22,
 'beta': 0.5,
 'alpha': 0.75,
 'n_trips_avg': 7238.375,
 'n_unfilled_demand_avg': 2434.25,
 'demand_avg': 9672.625,
 'n_parking_avg': 18295.625,
 'n_relocations_avg': 3710.0}

# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [18]:
capacities = [
    {
        "kick_scooter": 10000,
        "bicycle": 4000,
        "car": 3000,
    },
    {
        "kick_scooter": 8000,
        "bicycle": 3000,
        "car": 2000,
    },
    {
        "kick_scooter": 6000,
        "bicycle": 2000,
        "car": 1000,
    },
]


In [19]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.44 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.45 seconds
_set_max_demand finished in 0.12 seconds
Status: Optimal
Optimal Value of Objective Function:  27179.69136345392
Runtime without preprocessing: 44.24 seconds
solve finished in 75.17 seconds


Status: Optimal
Optimal Value of Objective Function:  27382.46934985442
Runtime without preprocessing: 37.63 seconds
solve finished in 74.13 seconds


Status: Optimal
Optimal Value of Objective Function:  -183163.81878168977
Runtime without preprocessing: 31.33 seconds
solve finished in 345.33 seconds


Status: Optimal
Optimal Value of Objective Function:  -182933.44880661144
Runtime without preprocessing: 27.64 seconds
solve finished in 75.06 seconds


Status: Optimal
Optimal Value of Objective Function:  31639.020380115195
Runtime without preprocessing: 46.45 seconds
solve finished

In [20]:
results_df = pd.DataFrame.from_dict(results)

In [21]:
results_df

,status,objective,solver_runtime,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,27179.691363,44.243824,7929.8125,1585.8125,9515.625,0.0000,26070.1875,10000,4000,3000,False,False
1,Optimal,27382.469350,37.627232,7940.1875,1575.4375,9515.625,0.0000,26059.8125,10000,4000,3000,False,True
2,Optimal,-183163.818782,31.332453,5360.1250,4155.5000,9515.625,28639.8750,0.0000,10000,4000,3000,True,False
3,Optimal,-182933.448807,27.642780,5362.3125,4153.3125,9515.625,28637.6875,0.0000,10000,4000,3000,True,True
4,Optimal,31639.020380,46.452361,7590.1250,1925.5000,9515.625,0.0000,18409.8750,8000,3000,2000,False,False
5,Optimal,31888.696834,39.118114,7602.2500,1913.3750,9515.625,0.0000,18397.7500,8000,3000,2000,False,True
6,Optimal,-121956.731577,31.810489,4568.3750,4947.2500,9515.625,21431.6250,0.0000,8000,3000,2000,True,False
7,Optimal,-121888.438457,27.302864,4569.2500,4946.3750,9515.625,21430.7500,0.0000,8000,3000,2000,True,True
8,Optimal,33268.265348,46.304055,7095.3750,2420.2500,9515.625,0.0000,10904.6250,6000,2000,1000,False,False
9,Optimal,33554.975601,38.995524,7147.8750,2367.7500,9515.625,0.0000,10852.1250,6000,2000,1000,False,True


In [22]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [23]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand_unstacked
last_demand: pd.DataFrame = (
    last_demand_unstacked["demand"]["car"].to_frame().stack().to_frame(name="demand")
)
last_demand.index = last_demand.index.set_names(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)


In [24]:
results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    current_fleet_capacity = {
        current_vehicle_type: real_fleet_size[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(
            lambda x: current_vehicle_type
        ),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


Status: Optimal
Optimal Value of Objective Function:  12417.529753590632
Runtime without preprocessing: 13.14 seconds
solve finished in 19.56 seconds
Status: Optimal
Optimal Value of Objective Function:  1782.9972705589685
Runtime without preprocessing: 13.04 seconds
solve finished in 19.16 seconds
Status: Optimal
Optimal Value of Objective Function:  7636.606965523751
Runtime without preprocessing: 13.12 seconds
solve finished in 18.77 seconds


In [25]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(real_fleet_size)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  31639.020380115195
Runtime without preprocessing: 44.20 seconds
solve finished in 72.78 seconds


In [26]:
results = pd.DataFrame.from_dict(results)

In [27]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,solver_runtime,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,31639.02038,44.197547,7590.1250,1925.500,9515.6250,0.0,18409.8750,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,21837.13399,39.295919,7720.1875,13571.375,21291.5625,0.0,18279.8125,['car']['bicycle']['kick_scooter']


In [43]:
os.makedirs(os.path.dirname(PATH_RESULTS_SINGLE_MODAL_BENCHMARK), exist_ok=True)
compare.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)

## Value Of The Stochastic Solution

In [28]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [30]:
results = []
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  659.3184262029068
Runtime without preprocessing: 1.92 seconds
solve finished in 3.06 seconds
Status: Optimal
Optimal Value of Objective Function:  1101.9133428664495
Runtime without preprocessing: 1.95 seconds
solve finished in 3.45 seconds


In [31]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,solver_runtime,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,659.318426,1.918428,429.0,8545.0,8974.0,0.0,271.0,"[kick_scooter, bicycle, car]"
1,Optimal,1101.913343,1.945398,441.0,10422.0,10863.0,0.0,259.0,"[kick_scooter, bicycle, car]"


In [42]:
os.makedirs(os.path.dirname(PATH_RESULTS_SINGLE_MODAL_BENCHMARK), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)